# FreeSurfer to Stanford Hardi Label Converter

This script alignes a given T1 weighted images to a Diffusion weighted images. The advantage of this way of alignment is, that the bvecs don't need to be reorientated as well. Additionally this script maps the FreeSurfer labels to the Stanford HARDI labels.

In [ ]:
import nibabel as nib
import nipy
import numpy as np
import matplotlib.pyplot as plt
import nipy.algorithms.registration
import utils
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table
from parallelization import *
import tempfile
from utilities import *
import os

In [ ]:
# Aligns T1 image to b0 image and uses this tranformation matrix on the labels
# T1 and label image are aligned, and should be aligned to DWI image
def T1_alignment(t1_image, dwi_image_b0, label_image):
    print "Align and downsample T1 + labels to DWI image"
    
    # Filter White Matter, Corpus Callosum, ...
    label_data = label_image.get_data()
    freesurfer_label_list = [[2,41,77,85,1004,2004], # Label 1 -> White Matter
                             [251,252,253,254,255],  # Label 2 -> Corpus Callosum
                             [1032],                 # Label 3
                             [1014],                 # Label 4
                             [1012],                 # Label 5
                             [1019],                 # Label 6
                             [1020],                 # Label 7
                             [1018],                 # Label 8
                             [1027],                 # Label 9
                             [1003],                 # Label 10
                             [1028],                 # Label 11
                             [1024],                 # Label 12
                             [1017],                 # Label 13
                             [1035],                 # Label 14
                             [1026],                 # Label 15
                             [1002],                 # Label 16
                             [1023],                 # Label 17
                             [1010],                 # Label 18
                             [1006],                 # Label 19
                             [1007],                 # Label 20
                             [1016],                 # Label 21
                             [1009],                 # Label 22
                             [1033],                 # Label 23
                             [1015],                 # Label 24
                             [1030],                 # Label 25
                             [1034],                 # Label 26
                             [1001],                 # Label 27
                             [1022],                 # Label 28
                             [1031],                 # Label 29
                             [1008],                 # Label 30
                             [1029],                 # Label 31
                             [1025],                 # Label 32
                             [1005],                 # Label 33
                             [1011],                 # Label 34
                             [1021],                 # Label 35
                             [1013],                 # Label 36
                             [11],                   # Label 37
                             [12],                   # Label 38
                             [13],                   # Label 39
                             [9],                    # Label 40
                             [10],                   # Label 41
                             [17],                   # Label 42
                             [18],                   # Label 43
                             [26],                   # Label 44
                             [28],                   # Label 45
                             [2032],                 # Label 46
                             [2014],                 # Label 47
                             [2012],                 # Label 48
                             [2019],                 # Label 49
                             [2020],                 # Label 50
                             [2018],                 # Label 51
                             [2027],                 # Label 52
                             [2003],                 # Label 53
                             [2028],                 # Label 54
                             [2024],                 # Label 55
                             [2017],                 # Label 56
                             [2035],                 # Label 57
                             [2026],                 # Label 58
                             [2002],                 # Label 59
                             [2023],                 # Label 60
                             [2010],                 # Label 61
                             [2006],                 # Label 62
                             [2007],                 # Label 63
                             [2016],                 # Label 64
                             [2009],                 # Label 65
                             [2033],                 # Label 66
                             [2015],                 # Label 67
                             [2030],                 # Label 68
                             [2034],                 # Label 68
                             [2001],                 # Label 70
                             [2022],                 # Label 71
                             [2031],                 # Label 72
                             [2008],                 # Label 73
                             [2029],                 # Label 74
                             [2025],                 # Label 75
                             [2005],                 # Label 76
                             [2011],                 # Label 77
                             [2021],                 # Label 78
                             [2013],                 # Label 79
                             [50],                   # Label 80
                             [51],                   # Label 81
                             [52],                   # Label 82
                             [48],                   # Label 83
                             [49],                   # Label 84
                             [53],                   # Label 85
                             [54],                   # Label 86
                             [58],                   # Label 87
                             [60]]                   # Label 88
    
    number_categories = len(freesurfer_label_list)
    
    # assignment  of new labels
    # every new label category gets an own dimention (X,Y,Z, Label)
    rearanged_label_list = np.zeros(label_data.shape + (number_categories,)).astype(bool)
    for category in range(number_categories):
        for label in freesurfer_label_list[category]:
            rearanged_label_list[..., category] = np.logical_or(rearanged_label_list[..., category], (label_data == label))
    
    # Create an image for each label category, to use resampling function
    new_label_image = nib.Nifti1Image(rearanged_label_list.astype(float), label_image.affine, header=label_image.header)     
    new_label_images = nib.four_to_three(new_label_image)
    
    # Register T1 Image to DWI b0 Image and resample T1
    t1_image_squeezed = nib.squeeze_image(t1_image)
    reg = nipy.algorithms.registration.HistogramRegistration(dwi_image_b0, t1_image_squeezed, similarity='crl1', interp='tri')
    T = reg.optimize('rigid')
    t1 = nipy.algorithms.registration.resample(t1_image_squeezed, T, reference=dwi_image_b0, interp_order=1)

    # Resample Labels according to DWI Images
    # Each on it's own to prevent mistakes in translation/rounding interpolation
    # for example labels (0,2) shouldn't become (1) afterwards, so translation of each label neccessary
    print "Rotate each label seperately"
    p = parallelization()
    labels_interpolated_list = p.start(rotate, number_categories, range(number_categories), new_label_images, [dwi_image_b0], [T])
    
    print "Merge labels into single mask"
    labels_interpolated = np.asarray(labels_interpolated_list) 
    labels_interpolated = np.sum(labels_interpolated, axis=0)

    # Returns new labels image and T1 image, which are align with DWI image now
    return labels_interpolated.astype(int), t1

In [ ]:
def rotate(category, new_label_image, dwi_image_b0, T):
    image_temp = nipy.algorithms.registration.resample(new_label_image, T, reference=dwi_image_b0, interp_order=1)
    label_temp = np.round(image_temp.get_data()).astype(int)
    labels_interpolated = label_temp * (category+1)
    return labels_interpolated

In [ ]:
# Library of Files
path = '/hcp/'
path_saveing = '/data/hcp/data/'
label_path = '/T1w/aparc+aseg.nii.gz'
t1_path = '/T1w/T2w_acpc_dc_restore.nii.gz'

subjects = os.listdir(path)

#OverflowError: size does not fit in an int
subjects.remove('103515')

#IOError: [Errno 2] No such file or directory
subjects.remove('.snapshot')
subjects.remove('109325')  
subjects.remove('114924')
subjects.remove('116120')
subjects.remove('121315')
subjects.remove('126931') 
subjects.remove('128329')
subjects.remove('129432') 
subjects.remove('129533')
subjects.remove('131621')
subjects.remove('142424')
subjects.remove('143527')
subjects.remove('145531')
subjects.remove('165234')
subjects.remove('169141')
subjects.remove('197449')
subjects.remove('207628')
subjects.remove('355542')
subjects.remove('552544')
subjects.remove('611231')
subjects.remove('650746')
subjects.remove('745555') 
subjects.remove('782157') 

for subject in subjects:
    print 'Process subject ' + subject
    
    # load images and gtabs
    t1_image = nib.load(path + subject + t1_path)
    label_image = nib.load(path + subject + label_path)
    data, affine, gtab, header, shell_mask = load_hcp_data(path, subject)

    # calculate out of all b0 measurements of a voxel a mean b0 value for each (x,y,z)
    average_data = np.mean(data[..., gtab.b0s_mask], -1)
    average_image = nib.Nifti1Image(average_data, affine)

    # Run Mask Alignment
    mask, t1 = T1_alignment(t1_image, average_image, label_image)

    print "Save results"
    # Save motion corrected T1
    t1_save = nib.Nifti1Image(t1.get_data(), t1.affine)
    t1_new_file = path_saveing + subject + '/T2_downsampled.nii.gz'
    nib.save(t1_save, t1_new_file)

    # Save motion corrected mask
    to_nifti = nib.Nifti1Image(mask.astype(int), average_image.affine)
    white_matter_file = path_saveing + subject + '/mask_downsampled.nii.gz'
    nib.save(to_nifti, white_matter_file)